In [1]:
import selenium
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
driver=Chrome()
driver.get('https://www.amazon.in/Apple-iPhone-128GB-Space-Black/product-reviews/B0BDJ22G36/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

In [3]:
content=driver.find_element(By.XPATH,'//div[@id="cm_cr-review_list"]')

In [5]:
posting=content.find_elements(By.XPATH,'//div[@data-hook="review"]')

In [7]:
Name=[]
Title=[]
Reviews=[]

for i in posting:
    name=i.find_element(By.XPATH,'.//span[@class="a-profile-name"]').text
    title=i.find_element(By.XPATH,'.//a[@data-hook="review-title"]').text
    review=i.find_element(By.XPATH,'.//span[@data-hook="review-body"]').text
    
    Name.append(name)
    Title.append(title)
    Reviews.append(review)

In [8]:
Title

['Wonder full .thats so pretty',
 'Great',
 'A smooth machine',
 'Good and Genuine so far',
 'Purchased this only for “Dynamic Island”',
 'Best iPhone yet.',
 'battery got discharged very fast in about 14-16 hours while watching videos online',
 'Welcome to "Dynamic Island"',
 "Sound quality is very very good. It's nearer to Dolby.",
 'Value for money']

In [9]:
next_page=driver.find_element(By.XPATH,'//ul[@class="a-pagination"]/li[2]').click()

In [25]:
Name=[]
Title=[]
Reviews=[]

current_page=1
driver=Chrome()
driver.get('https://www.amazon.in/Apple-iPhone-128GB-Space-Black/product-reviews/B0BDJ22G36/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
sign_tab=driver.find_element(By.XPATH,'//*[@id="nav-link-accountList"]').click()
email=driver.find_element(By.XPATH,'//*[@id="ap_email"]')
email.send_keys('****')
email.send_keys(Keys.ENTER)

password=driver.find_element(By.XPATH,'//*[@id="ap_password"]')
password.send_keys('****')
password.send_keys(Keys.ENTER)


while current_page in range(1,21):
    time.sleep(1)
    content=driver.find_element(By.XPATH,'//div[@id="cm_cr-review_list"]')
    posting=content.find_elements(By.XPATH,'//div[@data-hook="review"]')
    
    for i in posting:
        name=i.find_element(By.XPATH,'.//span[@class="a-profile-name"]').text
        title=i.find_element(By.XPATH,'.//a[@data-hook="review-title"]').text
        review=i.find_element(By.XPATH,'.//span[@data-hook="review-body"]').text

        Name.append(name)
        Title.append(title)
        Reviews.append(review)
        
    try:
        next_page=driver.find_element(By.XPATH,'//ul[@class="a-pagination"]/li[2]').click()
    except:
        pass
    
    current_page+=1
    
data=pd.DataFrame({'Name':Name,'Title':Title,'Reviews':Reviews})

    
    
    



# Text_classification

In [27]:
#checking for null values and dropping them

data.dropna(inplace=True)

In [40]:
data

,Name,Title,Review
0,jitu,Wonder full .thats so pretty,Good
1,Mr. M,Great,Apple hasn’t made any bones about the fact tha...
2,sudhanshu,A smooth machine,Loved the phone
3,Bhanu,Good and Genuine so far,
4,AVI deOry,Purchased this only for “Dynamic Island”,"(Pre-ordered on 10th of September, package re..."
...,...,...,...
195,Himanshu Tripathi,AWESOME PHONE.,VERY NICE PHONE UNBELIEVABLE CAMERA UNBELIEVAB...
196,Vedmani,Badhiya hai,Camera iphone13promax se badhiya hai per itna ...
197,Jasim Khan,Love the Dynamic Island,Absolutely love the phone. Just to get the new...
198,Tarush Saxena,Good Phone overall,"Amazing phone, tho ios a bit laggy-ish but oka..."


In [ ]:
#checking for blanks

blanks=[]

for i,na,te,re in data.itertuples():  #iterate over the DataFrame
    if type(re)==str:                 # avoid NaN values
        if re.isspace():              # test 'review' for whitespace
            blanks.append(i)          # add matching index numbers to the list
print(blanks)

# Sentiment_Analysis



In [47]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid=SentimentIntensityAnalyzer()


Use sid to append a comp_score to the dataset

In [48]:
#here we are appying sid.polarity_scores(review) for evry observation in the data
data['score']=data['Review'].apply(lambda y:sid.polarity_scores(y))

In [52]:
data['compound']=data['score'].apply(lambda y:y['compound'])

In [58]:
data['Label']=data['compound'].apply(lambda y:'pos' if y>=0 else 'neg')

In [59]:
data

,Name,Title,Review,score,compound,Label
0,jitu,Wonder full .thats so pretty,Good,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4404,pos
1,Mr. M,Great,Apple hasn’t made any bones about the fact tha...,"{'neg': 0.023, 'neu': 0.806, 'pos': 0.17, 'com...",0.9999,pos
2,sudhanshu,A smooth machine,Loved the phone,"{'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'comp...",0.5994,pos
3,Bhanu,Good and Genuine so far,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0.0000,pos
4,AVI deOry,Purchased this only for “Dynamic Island”,"(Pre-ordered on 10th of September, package re...","{'neg': 0.043, 'neu': 0.841, 'pos': 0.116, 'co...",0.8706,pos
...,...,...,...,...,...,...
195,Himanshu Tripathi,AWESOME PHONE.,VERY NICE PHONE UNBELIEVABLE CAMERA UNBELIEVAB...,"{'neg': 0.0, 'neu': 0.689, 'pos': 0.311, 'comp...",0.7368,pos
196,Vedmani,Badhiya hai,Camera iphone13promax se badhiya hai per itna ...,"{'neg': 0.016, 'neu': 0.862, 'pos': 0.122, 'co...",0.9659,pos
197,Jasim Khan,Love the Dynamic Island,Absolutely love the phone. Just to get the new...,"{'neg': 0.062, 'neu': 0.653, 'pos': 0.285, 'co...",0.8901,pos
198,Tarush Saxena,Good Phone overall,"Amazing phone, tho ios a bit laggy-ish but oka...","{'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'comp...",0.9371,pos


# Topic Modelling

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
tfidf=TfidfVectorizer(max_df=.95,min_df=2,stop_words='english')

In [62]:
dtm=tfidf.fit_transform(data['Review'])

In [63]:
dtm

<200x460 sparse matrix of type '<class 'numpy.float64'>'
	with 5970 stored elements in Compressed Sparse Row format>

In [64]:
LDA=LatentDirichletAllocation(n_components=7,random_state=42)

In [65]:
LDA.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

In [66]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names_out()[i] for i in topic.argsort()[-10:]])
    print('\n')

THE TOP 10 WORDS FOR TOPIC #0
['night', 'phone', 'price', 'buy', 'cool', 'bye', 'simple', 'don', 'money', 'waste']


THE TOP 10 WORDS FOR TOPIC #1
['ios', 'pro', 'awesome', 'amazing', 'best', 'phone', 'iphone', 'max', 'absolutely', 'bit']


THE TOP 10 WORDS FOR TOPIC #2
['mobile', 'totally', 'good', 'device', 'nice', 'improved', 'phone', 'iphone', 'unbelievable', 'features']


THE TOP 10 WORDS FOR TOPIC #3
['amazon', 'works', 'charge', 'good', 'days', 'camera', 'obviously', 'got', 'bought', 'apple']


THE TOP 10 WORDS FOR TOPIC #4
['upgrade', 'excellent', 'doesn', 'round', 'consider', 'chin', 'watching', 'youtube', 'good', '13']


THE TOP 10 WORDS FOR TOPIC #5
['13', 'hua', 'badhiya', 'mode', 'se', 'upgrade', 'heavy', 'nhi', 'hai', 'pro']


THE TOP 10 WORDS FOR TOPIC #6
['best', 'life', 'huge', 'power', 'level', 'bionic', 'a16', 'smoother', 'godly', 'loved']




In [67]:
topic_results=LDA.transform(dtm)

In [68]:
data['Topic']=topic_results.argmax(axis=1)

In [69]:
data

,Name,Title,Review,score,compound,Label,Topic
0,jitu,Wonder full .thats so pretty,Good,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.4404,pos,4
1,Mr. M,Great,Apple hasn’t made any bones about the fact tha...,"{'neg': 0.023, 'neu': 0.806, 'pos': 0.17, 'com...",0.9999,pos,1
2,sudhanshu,A smooth machine,Loved the phone,"{'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'comp...",0.5994,pos,6
3,Bhanu,Good and Genuine so far,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",0.0000,pos,0
4,AVI deOry,Purchased this only for “Dynamic Island”,"(Pre-ordered on 10th of September, package re...","{'neg': 0.043, 'neu': 0.841, 'pos': 0.116, 'co...",0.8706,pos,1
...,...,...,...,...,...,...,...
195,Himanshu Tripathi,AWESOME PHONE.,VERY NICE PHONE UNBELIEVABLE CAMERA UNBELIEVAB...,"{'neg': 0.0, 'neu': 0.689, 'pos': 0.311, 'comp...",0.7368,pos,2
196,Vedmani,Badhiya hai,Camera iphone13promax se badhiya hai per itna ...,"{'neg': 0.016, 'neu': 0.862, 'pos': 0.122, 'co...",0.9659,pos,5
197,Jasim Khan,Love the Dynamic Island,Absolutely love the phone. Just to get the new...,"{'neg': 0.062, 'neu': 0.653, 'pos': 0.285, 'co...",0.8901,pos,1
198,Tarush Saxena,Good Phone overall,"Amazing phone, tho ios a bit laggy-ish but oka...","{'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'comp...",0.9371,pos,1


# Text Classification

In [70]:
from sklearn.model_selection import train_test_split
X=data['Review']
Y=data['Label']

x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=.30,random_state=42)

In [71]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

text_clf=Pipeline([('tfidf',TfidfVectorizer()),
                  ('clf',MultinomialNB())])

In [72]:
text_clf.fit(x_train,y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [73]:
predict=text_clf.predict(x_test)

In [74]:
from sklearn.metrics import classification_report
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

         neg       1.00      0.55      0.71        11
         pos       0.91      1.00      0.95        49

    accuracy                           0.92        60
   macro avg       0.95      0.77      0.83        60
weighted avg       0.92      0.92      0.91        60

